Tuodaan tarvittavat kirjastot / Import necessary Python libraries:

In [ ]:
import os
import glob
import pathlib
from PyPDF2 import PdfFileMerger
import re
from reportlab.lib import colors
from reportlab.lib.styles import ParagraphStyle as PS
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image, PageBreak, Table, TableStyle
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.platypus.doctemplate import Frame
from reportlab.platypus.tableofcontents import TableOfContents
from reportlab.lib.pagesizes import A1, A2, A3, A4, landscape, portrait
from reportlab.lib.units import cm
import string
import datetime
from time import strftime

Yhdistetään pdf-tiedostot yhdeksi pdf-tiedostoksi: / Let's merge multiple pdf files together:

In [ ]:
class MyDocTemplate(BaseDocTemplate):
    
    def __init__(self, filename, **kw):
        self.allowSplitting = 0
        BaseDocTemplate.__init__(self, filename, **kw)
        template = PageTemplate('normal', [Frame(2.5*cm, 2.5*cm, 37*cm, 25*cm, id='F1')], pagesize = landscape(A3))
        self.addPageTemplates(template)

    def afterFlowable(self, flowable):
        "Registers TOC entries."
        if flowable.__class__.__name__ == 'Paragraph':
            text = flowable.getPlainText()
            style = flowable.style.name
            if style == 'Heading1':
                self.notify('TOCEntry', (0, text, self.page))
            if style == 'Heading2':
                self.notify('TOCEntry', (1, text, self.page))

h_title = PS(name = 'h_title', fontSize = 36, leading = 18)
h0 = PS(name = 'Heading0', fontSize = 14, leading = 16)
h1 = PS(name = 'Heading1', fontSize = 14, leading = 16)
h2 = PS(name = 'Heading2', fontSize = 12, leading = 14, leftIndent = 1*cm)
h3 = PS(name = 'Heading3', fontSize = 12, leading = 14, leftIndent = 1*cm)
# ------------------------------------------------------------------------------------------
def create_pdf(fname, fpaths_method_figures, figures_df, front_cover, back_cover, number_of_visit_at_the_reference_point, number_of_visit_at_the_reference_point_folder_path, encrypt_pswd):
    filename = '{0}{3}{1}{2}'.format(number_of_visit_at_the_reference_point_folder_path, fname, '.pdf', os.sep)
    doc = SimpleDocTemplate(filename, rightMargin = 0, leftMargin= 0, topMargin = 0, bottomMargin= 0)
    Story = []
    toc = TableOfContents()
    width = 12.5*cm
    height = 6*cm    
    toc.levelStyles = [h1, h2]
    Story.append(Paragraph('This PDF document was produced:', PS('body')))
    now1 = datetime.datetime.now()
    now1 = now1.strftime('%Y-%m-%d %H:%M:%S')
    now2 = str(datetime.date.today())
    Story.append(Paragraph(now1, PS('body')))
    Story.append(Paragraph('<br/><br/><br/><br/><br/><br/><br/>NLS GNSS SOFAMESA<br/><br/>', h_title))
    # Story.append(Paragraph('GNSS Position Solution Analysis Software of the National Land Survey of Finland<br/>Beta Version 0.2<br/>NLS repository: https://github.com/nlsfi/nls-gnss-sofamesa<br/>Updated repository: https://github.com/dmm809/nls-gnss-sofamesa<br/>{}<br/><br/><br/><br/><br/><br/><br/>'.format(now2), PS('body')))
    Story.append(Paragraph('GNSS Position Solution Analysis Software of the National Land Survey of Finland<br/>Beta Version 0.2<br/>{}<br/><br/>NLS repository: https://github.com/nlsfi/nls-gnss-sofamesa<br/>Updated repository: https://github.com/dmm809/nls-gnss-sofamesa<br/><br/><br/><br/><br/><br/><br/>'.format(now2), PS('body')))
    Story.append(Paragraph('Combination of the <br/><br/>Measurement Reports<br/>', h_title))
    Story.append(Paragraph('<br/><br/>Date: {0}<br/>Reference point: {1}<br/>Number of visit: {2}<br/><br/><br/><br/><br/>'.format(testmeasurements_df['Date'].iloc[0], name_of_the_reference_point_folder, number_of_visit_at_the_reference_point), PS('body')))
    fc_image = Image(front_cover, width, height)
    fc_image.hAlign = 'LEFT'
    Story.append(fc_image)
    # credits = ('\nAuthor: Tuukka Mattila.\nDo not hesitate to contact via LinkedIn: https://www.linkedin.com/in/tuukkamattila/.\n\nThank you for the help: Topi Rikkinen, Marko Ollikainen, Antti Laaksonen, Hannu Koivula, Octavian Andrei, Mikael Kauhava, Timo Sallinen and Ari Huvinen.').replace('\n','<br />\n')
    # credits = (' https://github.com/nlsfi/nls-gnss-sofamesa ')
    # Story.append(Paragraph(credits, PS('body')))
    Story.append(PageBreak())
    Story.append(toc)
    Story.append(PageBreak())
    Story.append(Paragraph('Combination of the Measurement Reports <br/><br/>', h1))
    Story.append(Paragraph('Please find the combined measurement results, which NLS GNSS SOFAMESA produces, on the upcoming pages.', PS('body')))
    Story.append(PageBreak())
    Story.append(Paragraph('Statistics Behind the Figures <br/><br/>', h1))
    tb1 = Table(np.vstack((list(all_table1s), np.array(all_table1s))).tolist())
    tb1style = TableStyle([('FONT', (0, 0), (-1, 0), 'Helvetica-Bold', 12),
                           ('FONT', (0, 1), (0, -1), 'Helvetica-Bold'),
                           ('GRID', (0, 0), (-1,-1), 0.5, colors.blue),
                           ('LINEABOVE', (0, 0), (-1, 0), 2.5, colors.blue),
                           ('ALIGN', (-1, 0), (-1, -1), 'LEFT'),
                          ('LINEBELOW', (0, 0), (-1, 0), 2.5, colors.blue),
                          ('LINEBEFORE', (0, 0), (0, -1), 2.5, colors.blue),
                           ('LINEAFTER', (-1, 0), (-1, -1), 2.5, colors.blue)])
    tb1.setStyle(tb1style)
    tb2 = Table(np.vstack((list(all_table2s), np.array(all_table2s))).tolist())
    tb2style = TableStyle([('FONT', (0, 0), (-1, 0), 'Helvetica-Bold', 12),
                           ('FONT', (0, 1), (0, -1), 'Helvetica-Bold'),
                           ('GRID', (0, 0), (-1,-1), 0.5, colors.green),
                           ('LINEABOVE', (0, 0), (-1, 0), 2.5, colors.green),
                           ('ALIGN', (-2, 0), (-1, -1), 'LEFT'),
                          ('LINEBELOW', (0, 0), (-1, 0), 2.5, colors.green),
                          ('LINEBEFORE', (0, 0), (0, -1), 2.5, colors.green),
                           ('LINEAFTER', (-1, 0), (-1, -1), 2.5, colors.green)])
    tb2.setStyle(tb2style)
    Story.append(tb1)
    Story.append(Paragraph('<br/><br/>', PS('body')))
    Story.append(PageBreak())
    Story.append(tb2)
    Story.append(Paragraph('<br/><br/>', PS('body')))
    Story.append(Paragraph('* The precision of the measurements = inner accuracy.<br/><br/>', PS('body')))
    Story.append(Paragraph('** The accuracy of the measurements = outer accuracy.<br/><br/>', PS('body')))
    Story.append(Paragraph('*** Horizontal RMSE is 63-68% of the measurements and it depends on the shape of the distribution. Horizontal 2dRMSE is 95-98% of the measurements and it also depends on the shape of the distribution.<br/><br/>', PS('body')))
    Story.append(Paragraph('**** Vertical RMSE is 63-68% of the measurements and it depends on the shape of the distribution. Vertical 2dRMSE is 95-98% of the measurements and it also depends on the shape of the distribution.<br/><br/>', PS('body')))
    Story.append(PageBreak())
    Story.append(Paragraph('Combination of the Figures <br/><br/>', h1))
    Story.append(Paragraph('Please find the combined figures of the measurement results, which NLS GNSS SOFAMESA produces, on the upcoming pages.', PS('body')))
    Story.append(PageBreak())
    method_name_number = 0
    index = 0
    while index < len(figures_df.index):
        fig_title = figures_df.loc[index, 'The Figure File Name'].split('.')[0]
        fig_title = fig_title.split('_')[1]
        Story.append(Paragraph('{}'.format(fig_title), h2))
        Story.append(Paragraph('<br/><br/>', PS('body')))
        tbl_figs1 = [[None]] * 1
        tbl_figs2 = [[None]] * 1
        while method_name_number < len(testmeasurements_name_list):
            row = figures_df.loc[index, 'The Figure File Name']
            width1 = 13.33*cm
            height1 = 10*cm
            while method_name_number < len(testmeasurements_name_list):
                def image_creation(fpaths_method_figures, method_name_number, row, width1, height1):
                    im = Image(pathlib.Path(fpaths_method_figures[method_name_number]).joinpath(row), width1, height1)
                    im.hAlign = 'LEFT'
                    return im
                if method_name_number < 3:
                        im = image_creation(fpaths_method_figures, method_name_number, row, width1, height1)
                        tbl_figs1[0].append([[im]])
                else:
                    im = image_creation(fpaths_method_figures, method_name_number, row, width1, height1)
                    tbl_figs2[0].append([[im]])
                break
            method_name_number += 1
        tbl_figs1 = Table(tbl_figs1)
        Story.append(tbl_figs1)
        tbl_figs2 = Table(tbl_figs2)
        Story.append(tbl_figs2)
        Story.append(PageBreak())
        method_name_number = 0
        index += 1
        # Figure 6 is not printed (Gaussian distributions)
        if index == 6:
            index += 1
    Story.append(Paragraph('Conclusions', h1))
    Story.append(PageBreak())
    Story.append(Paragraph('This PDF document was produced:', PS('body')))
    now1 = datetime.datetime.now()
    now1 = now1.strftime('%Y-%m-%d %H:%M:%S')
    Story.append(Paragraph(now1, PS('body')))
    Story.append(Paragraph('<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>NLS GNSS SOFAMESA<br/><br/>', h_title))
    # Story.append(Paragraph('GNSS Position Solution Analysis Software of the National Land Survey of Finland<br/>Beta Version 0.2<br/>NLS repository: https://github.com/nlsfi/nls-gnss-sofamesa<br/>Updated repository: https://github.com/dmm809/nls-gnss-sofamesa<br/>{}<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>'.format(now2), PS('body')))
    Story.append(Paragraph('GNSS Position Solution Analysis Software of the National Land Survey of Finland<br/>Beta Version 0.2<br/>{}<br/><br/>NLS repository: https://github.com/nlsfi/nls-gnss-sofamesa<br/>Updated repository: https://github.com/dmm809/nls-gnss-sofamesa<br/><br/><br/><br/><br/><br/><br/>'.format(now2), PS('body')))
    bc_image = Image(back_cover, width, height)
    bc_image.hAlign = 'LEFT'
    Story.append(bc_image)
    # Story.append(Paragraph(credits, PS('body')))
    
    doc = MyDocTemplate(filename,
                        title = 'The number of visit: {} analyzed by NLS GNSS SOFAMESA: GNSS Position Solution Analysis Software of the National Land Survey of Finland'.format(number_of_visit_at_the_reference_point),
                        # author = 'Tuukka Mattila. Do not hesitate to contact via LinkedIn: https://www.linkedin.com/in/tuukkamattila/',
                        encrypt = encrypt_pswd)
    doc.multiBuild(Story)
        
    print("{} created".format(filename))
    return filename

#----------------------------------------------------------------------
if __name__ == "__main__":
    path_pdf_accessories = pathlib.Path(pathlib.Path(pathlib.Path.cwd()).joinpath("for_the_pdf_report"))
    front_cover = os.path.join(path_pdf_accessories, "NLS_logo1.jpg")
    back_cover = os.path.join(path_pdf_accessories, "NLS_logo2.jpg")
    filename = '{0}_{1}_Combined_Measurement_Report'.format(name_of_the_reference_point_folder, number_of_visit_at_the_reference_point)
    encrypt_pswd = None
# encrypt_doc = input("Do you want to set up a password for the combination of the measurement reports .pdf file? Enter 'yes' or 'no': ")
# if encrypt_doc == 'yes':
#  encrypt_pswd = input("Enter a password for the combination of the measurement reports .pdf file: ")
#  filename_path = create_pdf(filename, front_cover, back_cover, number_of_visit_at_the_reference_point, number_of_visit_at_the_reference_point_folder_path, encrypt_pswd)
# elif encrypt_doc == 'no':
#  encrypt_pswd = None
#  filename_path = create_pdf(filename, front_cover, back_cover, number_of_visit_at_the_reference_point, number_of_visit_at_the_reference_point_folder_path, encrypt_pswd)
    filename_path = create_pdf(filename, filepaths_method_figures, figures_df, front_cover, back_cover, number_of_visit_at_the_reference_point, number_of_visit_at_the_reference_point_folder_path, encrypt_pswd)
#----------------------------------------------------------------------        
# def merger1(output_path, input_paths):
    # pdf_merger = PdfFileMerger()
    # file_handles = []
    
    # for path in input_paths:
        # pdf_merger.merge(position = 1, fileobj = path, pages = (6, 7))
        
    # with open(output_path, 'wb') as fileobj:
        # pdf_merger.write(fileobj)
    
    # output_path = [output_path]
    # return output_path

# output_filepaths = merger1('{0}{3}{1}_{2}_Combined_Measurement_Report_plain.pdf'.format(number_of_visit_at_the_reference_point_folder_path, name_of_the_reference_point_folder, number_of_visit_at_the_reference_point, os.sep), filepaths)
# output_filepaths.insert(0, filename_path)

# def merger2(output_path, input_paths):
    # pdf_merger = PdfFileMerger()
    # file_handles = []
    
    # for path in input_paths:
        # pdf_merger.merge(position = 3, fileobj = path)
        
    # with open(output_path, 'wb') as fileobj:
        # pdf_merger.write(fileobj)
    
    # output_path = [output_path]
    # return output_path

# output_filepaths_combined = merger2('{0}{3}{1}_{2}_Combined_Measurement_Report.pdf'.format(number_of_visit_at_the_reference_point_folder_path, name_of_the_reference_point_folder, number_of_visit_at_the_reference_point, os.sep), output_filepaths)